In [ ]:
import math
import numpy as np
import os
import open3d as o3d
import pprint

from evo.tools import plot
from evo.tools.plot import PlotMode
from evo.core.metrics import PoseRelation, Unit
from evo.tools.settings import SETTINGS
from evo.tools import file_interface
from evo.core import sync
from evo.core.sync import TrajectoryPair

from os import path
from tools.tum_plots import *
from tools.tum_tools import *

# temporarily override some package settings
SETTINGS.plot_figsize = [6, 6]
SETTINGS.plot_split = True
SETTINGS.plot_usetex = True

%load_ext autoreload
%autoreload 2

# magic plot configuration
import matplotlib.pyplot as plt
%matplotlib widget


#### Analyse Timestamps between PC and EKF Pose

In [ ]:
data_root = "data/ekf_analysis/v2/02-full-merge-ekf-feedback/run_01/"

gps_pose_name = "ekf_gt_gps.tum"
ekf_pose_name = "ekf_pose_with_covariance.tum"
kiss_pose_name = "odometry.tum"

# gps_file = path.join(data_root, gps_pose_name)
ekf_file = path.join(data_root, ekf_pose_name)
traj_ekf = file_interface.read_tum_trajectory_file(ekf_file)

# Read lidar timestamps
pcd_root = "/home/julian/projects/thesis/datasets/ext/converted/"
times_file = path.join(pcd_root, "mdv3_full_merge", "times/lidar_timestamps.txt")

stamps = np.loadtxt(times_file)

timestamps = stamps[:,0]  + stamps[:,1] *1e-9
ekf_stamps = traj_ekf.timestamps
timestamps = timestamps[:, np.newaxis]
ekf_stamps = ekf_stamps[np.newaxis, :]

differences = np.abs(timestamps- ekf_stamps)

closest_indices = np.argmin(differences, axis=1)

print(differences.shape)
print(closest_indices.shape)

closest_ekf_stamps = ekf_stamps[0, closest_indices]

differences = abs(timestamps[:,0] - closest_ekf_stamps)

x = differences
x, y = sorted(x), np.arange(len(x)) / len(x)
simple_plot(x,y, plot_fn=plt.plot, xlabel="Time Difference [s]", ylabel='CDF', label="timestamp_analysis.")

## Analyze PCs

In [ ]:
pcd_dir = path.join(pcd_root, "mdv3_full_merge", 'pcd')
pcd_files = [path.join(pcd_dir, f) for f in os.listdir(pcd_dir) if f.endswith('.pcd')]

In [ ]:
# Read pointcloud
pcd = o3d.io.read_point_cloud(pcd_files[0])

In [67]:
o3d.visualization.draw_geometries([pcd])

In [ ]:
print(pcd)

### Downsample

In [65]:
downpcd = pcd.voxel_down_sample(voxel_size=0.05)

In [66]:
o3d.visualization.draw_geometries([downpcd])